In [1]:
import pandas as pd
import os
import psycopg2
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

True

In [2]:
# Cria conexão com o banco de dados
def conexao():
    con = psycopg2.connect(
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD"),
        database=os.getenv("POSTGRES_DB"),
        host="localhost",
    )

    return con

# Cria função para selecionar os dados
def se_dados(query):
    vcon = conexao()
    c = vcon.cursor()
    c.execute(query)
    rows = c.fetchall()
    vcon.commit()
    vcon.close()
    return rows                    

In [4]:
# Cria query para gerar o relatório
query = f" SELECT DISTINCT papel, div_liquida, cotacao, pl, vpa, ev_ebitda, marg_liquida, lucro_liquido_12m, lucro_liquido_3m \
            FROM dados \
                WHERE data_dado_inserido >= '01-11-2021' \
                    AND div_liquida < '0' \
                        ORDER BY 2 "
result = se_dados(query)

# Cria DataFrame e salva o arquivo gerado em csv
df_rl = pd.DataFrame(result)
# Renomea o nome das colunas do Dataframe
df_rl.rename(columns={0:'Papel', 
    1:'Dívida Líquida', 
    2:'Cotação',
    3:'PL',
    4:'VPA',
    5:'EV/Ebitda',
    6:'Margem Líquida',
    7:'Lucro Líquido 12m',
    8:'Lucro Líquido 3m'}, inplace=True)
df_rl.to_csv('../Analise/relatorio.csv', sep=';', encoding='ISO-8859-1')
df_rl.head(7)

,Papel,Dívida Líquida,Cotação,PL,VPA,EV/Ebitda,Margem Líquida,Lucro Líquido 12m,Lucro Líquido 3m
0,ABEV3,"-R$ 16.973.000.000,00","R$ 17,36",17.24,5.51,12.39,23.4,"R$ 15.849.700.000,00","R$ 3.552.530.000,00"
1,SULA11,"-R$ 14.945.100.000,00","R$ 25,96",5.97,18.87,-6.31,9.7,"R$ 1.852.970.000,00","R$ 29.339.000,00"
2,SULA3,"-R$ 14.945.100.000,00","R$ 8,90",6.14,6.29,-5.80,9.7,"R$ 1.852.970.000,00","R$ 29.339.000,00"
3,SULA4,"-R$ 14.945.100.000,00","R$ 8,60",5.93,6.29,-6.42,9.7,"R$ 1.852.970.000,00","R$ 29.339.000,00"
4,PSSA3,"-R$ 9.319.390.000,00","R$ 23,40",8.61,14.43,0.36,8.9,"R$ 1.758.190.000,00","R$ 656.670.000,00"
5,B3SA3,"-R$ 8.281.460.000,00","R$ 11,98",15.67,3.70,8.49,45.3,"R$ 4.683.740.000,00","R$ 1.193.340.000,00"
6,CMIN3,"-R$ 5.897.070.000,00","R$ 5,38",3.92,2.77,1.67,34.2,"R$ 7.672.770.000,00","R$ 2.500.640.000,00"


In [5]:
import win32com.client as win32
import pathlib

# Inserindo arquivo a ser anexado
anexo = pathlib.Path('../Analise/relatorio.csv')
anexo_ab = str(anexo.absolute())

# Criar a integração com o outlook
outlook = win32.Dispatch('outlook.application')

# Criar um email
email = outlook.CreateItem(0)

# Configurar as informações do seu e-mail
email.To = os.getenv("EMAIL")
email.Subject = " Relatório - Ações listadas na Bolsa "
email.HTMLBody = f"""

<p>Olá,</p>
<br>
<p>Segue em anexo o relatório solicitado.</p>
<br>
<p>Atenciosamente,</p>
<p>Lucas Alves.</p>
<p>Analista de Dados</p>
<br>
<img src="../Image/1-relatorio-gerencial.jpg">

"""

# Configurar anexo
email.Attachments.Add(anexo_ab)

# Envia o e-mail
email.Send()
print("Email Enviado")

Email Enviado
